In [1]:
import getpass
import os     
from dotenv import load_dotenv
from langchain_openai import ChatOpenAI
import os

# This loads your API keys from a hidden .env file so they aren't exposed in the code.
load_dotenv()
from phoenix.otel import register

tracer_provider = register(auto_instrument=True)

/opt/anaconda3/lib/python3.12/site-packages/pydantic/_internal/_generate_schema.py:2249: UnsupportedFieldAttributeWarning: The 'validate_default' attribute with value True was provided to the `Field()` function, which has no effect in the context it was used. 'validate_default' is field-specific metadata, and can only be attached to a model field using `Annotated` metadata or by assignment. This may have happened because an `Annotated` type alias using the `type` statement was used, or if the `Field()` function was attached to a single member of a union type.
  warnings.warn(
Overriding of current TracerProvider is not allowed


🔭 OpenTelemetry Tracing Details 🔭
|  Phoenix Project: default
|  Span Processor: SimpleSpanProcessor
|  Collector Endpoint: localhost:4317
|  Transport: gRPC
|  Transport Headers: {}
|  
|  Using a default SpanProcessor. `add_span_processor` will overwrite this default.
|  
|  ⚠️ WARNING: It is strongly advised to use a BatchSpanProcessor in production environments.
|  
|  `register` has set this TracerProvider as the global OpenTelemetry default.
|  To disable this behavior, call `register` with `set_global_tracer_provider=False`.



In [1]:
import os
from crewai import Agent, Crew, Process, Task
from crewai_tools import SerperDevTool
import sys
# This disables the CrewAI instrumentation wrapper entirely
sys.modules["openinference.instrumentation.crewai"] = None
# -------------------------------------------------------------------------
# 1. SETUP
# -------------------------------------------------------------------------
# We instantiate the search tool. This allows agents to access the internet.
# Make sure you have SERPER_API_KEY and OPENAI_API_KEY in your environment variables.
search_tool = SerperDevTool()

# We define the LLM. You can change this to "gpt-3.5-turbo" or "ollama/mistral"
# depending on your budget and local setup.
llm_model = "gpt-4o-mini"

def create_research_crew(topic):
    """
    Creates a crew of agents to analyze a given topic.
    """

    # ---------------------------------------------------------------------
    # 2. AGENTS (The "Who")
    # ---------------------------------------------------------------------
    # IMPROVEMENT: The Backstories now give specific "personality traits"
    # that affect output style (e.g., "skeptical", "engaging", "concise").

    researcher = Agent(
        role="Senior Tech Research Analyst",
        # GOAL: Be specific. Not just "find info", but "find quantifiable info".
        goal=f"Uncover verified, quantifiable data and recent papers regarding {topic}.",
        backstory=(
            "You are a dogged researcher at a top-tier tech think tank. "
            "You do not settle for surface-level news. You dig for white papers, "
            "GitHub repositories, and primary sources. You prioritize facts over hype."
        ),
        verbose=True,
        allow_delegation=False, # We want this agent to do the work, not ask others.
        tools=[search_tool],
        llm=llm_model,
    )

    fact_checker = Agent(
        role="red-teaming Specialist / Fact Checker",
        # GOAL: This agent acts as a filter/quality gate.
        goal=f"Strictly audit the research findings on {topic} to prevent hallucinations.",
        backstory=(
            "You are a skepticism engine. Your job is to protect the organization's reputation "
            "by ensuring nothing false gets published. You cross-reference claims "
            "and flag vague statements. If a claim lacks a source, you reject it."
        ),
        verbose=True,
        allow_delegation=False,
        tools=[search_tool], # Needs tools to verify if the Researcher told the truth.
        llm=llm_model,
    )

    writer = Agent(
        role="Lead Tech Journalist",
        # GOAL: Focus on narrative flow and readability.
        goal=f"Synthesize verified facts into a narrative about {topic} that is easy to read.",
        backstory=(
            "You are a writer for a publication like 'The Verge' or 'Wired'. "
            "You despise corporate jargon. You write in short, punchy sentences. "
            "You use the verified data provided to you to tell a story, not just list facts."
        ),
        verbose=True,
        allow_delegation=False,
        llm=llm_model,
    )

    # ---------------------------------------------------------------------
    # 3. TASKS (The "What")
    # ---------------------------------------------------------------------
    # IMPROVEMENT: The descriptions now use a "Chain of Thought" style,
    # telling the agent exactly what steps to take.

    conduct_analysis_task = Task(
        description=(
            f"1. Search for the absolute latest advancements in {topic}. "
            "2. Identify at least 5 distinct trends or technologies. "
            "3. For each trend, find the specific company or research lab responsible. "
            "4. Collect URLs for every claim you make."
        ),
        # EXPECTED OUTPUT: specific formatting prevents the LLM from rambling.
        expected_output="A structured list of 5 advancements, each with a Title, Description, Source URL, and Date.",
        context=[],
        agent=researcher,
    )

    fact_checking_task = Task(
        description=(
            f"1. Review the list of advancements provided by the Researcher. "
            "2. Use your search tool to verify that these technologies actually exist "
            "and are not just rumors. "
            "3. If a claim is exaggerated (e.g., 'AGI is here'), tone it down to reality. "
            "4. Remove any items that lack credible sources."
        ),
        expected_output="A vetted, 'Green-Lit' list of facts, ready for publication. Mark rejected items as 'REJECTED'.",
        agent=fact_checker,
        # CONTEXT: This is crucial. It forces this task to wait for and use the Researcher's output.
        context=[conduct_analysis_task],
    )

    writer_task = Task(
        description=(
            f"Using ONLY the 'Green-Lit' facts provided by the Fact Checker, write a blog post about {topic}. "
            "1. Create a catchy, click-worthy title. "
            "2. Write a 100-word introduction setting the scene. "
            "3. Dedicate one short paragraph to each verified advancement. "
            "4. End with a 'Why this matters' section. "
            "5. Do NOT use the words 'delve', 'tapestry', or 'plethora'. Write like a human."
        ),
        expected_output="A markdown-formatted blog post, approx 600 words.",
        agent=writer,
        # CONTEXT: The writer ignores the researcher and listens only to the fact checker.
        context=[fact_checking_task],
    )

    # ---------------------------------------------------------------------
    # 4. CREW (The Orchestration)
    # ---------------------------------------------------------------------
    crew = Crew(
        agents=[researcher, fact_checker, writer],
        tasks=[conduct_analysis_task, fact_checking_task, writer_task],
        # Process.sequential ensures Task 1 finishes before Task 2 starts.
        process=Process.sequential, 
        verbose=True
    )

    return crew

# -------------------------------------------------------------------------
# 5. EXECUTION
# -------------------------------------------------------------------------
if __name__ == "__main__":
    # This block only runs if you execute this file directly.
    print("## Starting the Research Crew...")
    
    # Create the crew
    my_crew = create_research_crew("Agentic AI Security Standards")
    
    # Kickoff the process
    result = my_crew.kickoff()
    
    print("\n\n########################")
    print("## FINAL RESULT ##")
    print("########################\n")
    print(result)

## Starting the Research Crew...


╭──────────────────────────────────────────── Crew Execution Started ─────────────────────────────────────────────╮
│                                                                                                                 │
│  Crew Execution Started                                                                                         │
│  Name: crew                                                                                                     │
│  ID: 2c316eab-0c60-4ece-8635-ebea119eea07                                                                       │
│  Tool Args:                                                                                                     │
│                                                                                                                 │
│                                                                                                                 │
╰─────────────────────────────────────────────────────────────────────────────────────────────────────────────────╯

╭─────────────────────────────────────────────── 🤖 Agent Started ────────────────────────────────────────────────╮
│                                                                                                                 │
│  Agent: Senior Tech Research Analyst                                                                            │
│                                                                                                                 │
│  Task: 1. Search for the absolute latest advancements in Agentic AI Security Standards. 2. Identify at least 5  │
│  distinct trends or technologies. 3. For each trend, find the specific company or research lab responsible. 4.  │
│  Collect URLs for every claim you make.                                                                         │
│                                                                                                                 │
╰─────────────────────────────────────────────────────────────────────────────────────────────────────────────────╯

Output()

╭──────────────────────────────────────────── 🔧 Agent Tool Execution ────────────────────────────────────────────╮
│                                                                                                                 │
│  Agent: Senior Tech Research Analyst                                                                            │
│                                                                                                                 │
│  Thought: I need to search for the latest advancements in Agentic AI Security Standards to identify distinct    │
│  trends, the responsible companies or research labs, and gather relevant URLs for each finding.                 │
│                                                                                                                 │
│  Using Tool: Search the internet with Serper                                                                    │
│                                                                                                                 │
╰─────────────────────────────────────────────────────────────────────────────────────────────────────────────────╯

...

Output()

╭───────────────────────────────────────────── ✅ Agent Final Answer ─────────────────────────────────────────────╮
│                                                                                                                 │
│  Agent: Lead Tech Journalist                                                                                    │
│                                                                                                                 │
│  Final Answer:                                                                                                  │
│  # Seizing the Agentic AI Advantage                                                                             │
│                                                                                                                 │
│  In the high-stakes world of artificial intelligence, security is paramount. Agentic AI is not just a           │
│  buzzword; it’s a paradigm shift. As AI becomes more autonomous, the potential risks rise. To harness the       │
│  capabilities of AI responsibly, we need robust security frameworks. This article explores critical             │
│  advancements in Agentic AI Security Standards and their implications across various sectors. With the          │
│  adoption of these standards, we can face the challenges of this new technology head-on.                        │
│                                                                                                                 │
│  ### The GenAI Paradox                                                                                          │
│                                                                                                                 │
│  First up is the GenAI paradox. This phenomenon underscores a crucial final frontier: AI agents are only as     │
│  good as their programming. While they offer immense potential in various fields, such as healthcare and        │
│  finance, they also present unique operational vulnerabilities. Developing clearer guidelines and security      │
│  measures can help organizations capitalize on their benefits while minimizing risks. It’s a balancing act      │
│  that demands constant vigilance.                                                                               │
│                                                                                                                 │
│  ### The Agentic AI Security Scoping Matrix                                                                     │
│                                                                                                                 │
│  Next, let's discuss the Agentic AI Security Scoping Matrix developed by AWS. This framework identifies four    │
│  distinct risk categories for autonomous AI systems: operational, strategic, technical, and personal. By        │
│  categorizing these risks, AWS provides organizations with a structured way to assess and enhance their AI      │
│  security protocols. Tailoring security measures to each risk type makes proactive management possible,         │
│  ensuring that AI systems are not just intelligent but secure.                                                  │
│                                                                                                                 │
│  ### Navigating Top AI Security Threats                                                                         │
│                                                                                                                 │
│  Svitla Systems sheds light on the top threats facing agentic AI today. From data poisoning to adversarial      │
│  attacks, these threats are ever-evolving. The report outlines strategies to secure autonomous systems,         │
│  emphasizing the importance of continuous monitoring and adaptive defenses. As these technologies advance,      │
│  awareness of the potential risks can set the foundatio

╭──────────────────────────────────────────────── Task Completion ────────────────────────────────────────────────╮
│                                                                                                                 │
│  Task Completed                                                                                                 │
│  Name: 37587e41-d133-4561-8c24-d084289e003c                                                                     │
│  Agent: Lead Tech Journalist                                                                                    │
│  Tool Args:                                                                                                     │
│                                                                                                                 │
│                                                                                                                 │
╰─────────────────────────────────────────────────────────────────────────────────────────────────────────────────╯

╭──────────────────────────────────────────────── Crew Completion ────────────────────────────────────────────────╮
│                                                                                                                 │
│  Crew Execution Completed                                                                                       │
│  Name: crew                                                                                                     │
│  ID: 2c316eab-0c60-4ece-8635-ebea119eea07                                                                       │
│  Tool Args:                                                                                                     │
│  Final Output: # Seizing the Agentic AI Advantage                                                               │
│                                                                                                                 │
│  In the high-stakes world of artificial intelligence, security is paramount. Agentic AI is not just a           │
│  buzzword; it’s a paradigm shift. As AI becomes more autonomous, the potential risks rise. To harness the       │
│  capabilities of AI responsibly, we need robust security frameworks. This article explores critical             │
│  advancements in Agentic AI Security Standards and their implications across various sectors. With the          │
│  adoption of these standards, we can face the challenges of this new technology head-on.                        │
│                                                                                                                 │
│  ### The GenAI Paradox                                                                                          │
│                                                                                                                 │
│  First up is the GenAI paradox. This phenomenon underscores a crucial final frontier: AI agents are only as     │
│  good as their programming. While they offer immense potential in various fields, such as healthcare and        │
│  finance, they also present unique operational vulnerabilities. Developing clearer guidelines and security      │
│  measures can help organizations capitalize on their benefits while minimizing risks. It’s a balancing act      │
│  that demands constant vigilance.                                                                               │
│                                                                                                                 │
│  ### The Agentic AI Security Scoping Matrix                                                                     │
│                                                                                                                 │
│  Next, let's discuss the Agentic AI Security Scoping Matrix developed by AWS. This framework identifies four    │
│  distinct risk categories for autonomous AI systems: operational, strategic, technical, and personal. By        │
│  categorizing these risks, AWS provides organizations with a structured way to assess and enhance their AI      │
│  security protocols. Tailoring security measures to each risk type makes proactive management possible,         │
│  ensuring that AI systems are not just intelligent but secure.                                                  │
│                                                                                                                 │
│  ### Navigating Top AI Security Threats                                                                         │
│                                                                                                                 │
│  Svitla Systems sheds light on the top threats facing agentic AI today. From data poisoning to adversarial      │
│  attacks, these threats are ever-evolving. The report outlines strategies to secure autonomous systems,         │
│  emphasizing the importance of continuous monitoring a



########################
## FINAL RESULT ##
########################

# Seizing the Agentic AI Advantage

In the high-stakes world of artificial intelligence, security is paramount. Agentic AI is not just a buzzword; it’s a paradigm shift. As AI becomes more autonomous, the potential risks rise. To harness the capabilities of AI responsibly, we need robust security frameworks. This article explores critical advancements in Agentic AI Security Standards and their implications across various sectors. With the adoption of these standards, we can face the challenges of this new technology head-on.

### The GenAI Paradox

First up is the GenAI paradox. This phenomenon underscores a crucial final frontier: AI agents are only as good as their programming. While they offer immense potential in various fields, such as healthcare and finance, they also present unique operational vulnerabilities. Developing clearer guidelines and security measures can help organizations capitalize on their bene

In [11]:
import phoenix as px
px.launch_app()

import pandas as pd
import phoenix as px
import nest_asyncio
from phoenix.experiments import run_experiment

# Apply nested asyncio to allow running inside notebooks/scripts
nest_asyncio.apply()

# -------------------------------------------------------------------------
# 1. SETUP: Dynamic Data Generation (The "Golden Dataset")
# -------------------------------------------------------------------------

# The list of topics to test
topics = [
    "AI and data science advancements in 2024",
    "Quantum computing breakthroughs",
    "Climate tech innovations",
    "Biotechnology and gene editing progress",
    "Renewable energy solutions"
]

# We define the "Template" of what a correct run looks like.
# This ensures consistency without rewriting the same dictionary 5 times.
def generate_reference_trajectory(topic):
    return [
        {
            "tool_name": "Senior Research Analyst",
            "tool_input": f"Conduct comprehensive research on {topic}, analyzing key innovations, trends, and potential impacts.",
        },
        {
            "tool_name": "Specialist Fact Checker",
            "tool_input": f"Verify the accuracy of research findings on {topic}, ensuring all claims are supported by credible sources.",
        },
        {
            "tool_name": "Writer",
            "tool_input": f"Develop a comprehensive blog post about {topic}, incorporating verified research and balanced perspectives.",
        },
    ]

# Create the DataFrame programmatically
df = pd.DataFrame([
    {
        "topic": topic,
        "reference_trajectory": generate_reference_trajectory(topic)
    } 
    for topic in topics
])

# Initialize Phoenix Client and Dataset
phoenix_client = px.Client()
DATASET_NAME = "crewai-researcher-test-topics"

try:
    # Try to fetch existing, otherwise upload new
    dataset = phoenix_client.get_dataset(name=DATASET_NAME)
except ValueError:
    dataset = phoenix_client.upload_dataset(
        dataframe=df,
        dataset_name=DATASET_NAME,
        input_keys=["topic"],
        output_keys=["reference_trajectory"],
    )

# -------------------------------------------------------------------------
# 2. ADAPTER: Connect CrewAI to the Experiment
# -------------------------------------------------------------------------

def parse_crew_output(crew_output) -> list:
    """
    Parses the raw CrewOutput object into a list of dictionaries 
    matching our reference format.
    """
    actual_trajectory = []
    
    # CrewAI outputs contain a 'tasks_output' list
    if hasattr(crew_output, "tasks_output"):
        for task in crew_output.tasks_output:
            # Extract Agent Role (Handle both object and string cases)
            agent_role = "Unknown"
            if hasattr(task, "agent"):
                agent_role = getattr(task.agent, "role", str(task.agent))
            
            # Extract Description
            description = getattr(task, "description", str(task))
            
            actual_trajectory.append({
                "tool_name": agent_role, 
                "tool_input": description
            })
            
    return actual_trajectory

def task_wrapper(input_data):
    """
    The main function the experiment calls. 
    It runs the crew and returns the parsed trajectory.
    """
    # Assuming 'create_research_crew' is imported from your previous code
    # Note: We must ensure we pass the topic string correctly
    topic = input_data.get("topic")
    crew = create_research_crew(topic=topic)
    
    # Run the crew
    result = crew.kickoff()
    
    # Return the clean list of steps
    return parse_crew_output(result)

# -------------------------------------------------------------------------
# 3. EVALUATORS: Pure Python Metrics
# -------------------------------------------------------------------------

def score_agent_sequence(output, expected) -> float:
    """
    Checks if the Agents ran in the exact correct order.
    Returns 1.0 for perfect match, 0.0 otherwise.
    """
    predicted_traj = output  # Output is already parsed by task_wrapper
    reference_traj = expected.get("reference_trajectory", [])

    if len(predicted_traj) != len(reference_traj):
        return 0.0

    for i, step in enumerate(predicted_traj):
        # We check if the agent role matches
        if step["tool_name"] != reference_traj[i]["tool_name"]:
            return 0.0
            
    return 1.0

def score_input_similarity(output, expected) -> float:
    """
    Checks if the inputs (task descriptions) match strictly.
    (Note: In production, you might want fuzzy matching here).
    """
    predicted_traj = output
    reference_traj = expected.get("reference_trajectory", [])
    
    if not predicted_traj or len(predicted_traj) != len(reference_traj):
        return 0.0
        
    matches = 0
    for i, step in enumerate(predicted_traj):
        # normalize strings to avoid whitespace errors
        pred_desc = step["tool_input"].strip().lower()
        ref_desc = reference_traj[i]["tool_input"].strip().lower()
        
        # Simple containment check is often safer than exact equality for LLMs
        if pred_desc == ref_desc:
            matches += 1
            
    return matches / len(reference_traj)

# -------------------------------------------------------------------------
# 4. EXECUTION
# -------------------------------------------------------------------------

print("🧪 Starting Experiment...")

experiment = run_experiment(
    dataset=dataset,
    task=task_wrapper, # The function that runs the Crew
    experiment_name="agent-sequence-validation",
    evaluators=[
        score_agent_sequence,
        score_input_similarity
    ],
)

print(f"✅ Experiment Complete. View results at: {phoenix_client.url}")

📤 Uploading dataset...
💾 Examples uploaded: http://localhost:6006/datasets/RGF0YXNldDox/examples
🗄️ Dataset version ID: RGF0YXNldFZlcnNpb246MQ==
🧪 Starting Experiment...
🧪 Experiment started.
📺 View dataset experiments: http://localhost:6006/datasets/RGF0YXNldDox/experiments
🔗 View this experiment: http://localhost:6006/datasets/RGF0YXNldDox/compare?experimentId=RXhwZXJpbWVudDox


/var/folders/fk/np76hvhj1yj81k65qbkq0wv40000gn/T/ipykernel_18875/3778872193.py:55: DeprecationWarning: Migrate to using client.datasets.get_dataset via arize-phoenix-client
  dataset = phoenix_client.get_dataset(name=DATASET_NAME)
/var/folders/fk/np76hvhj1yj81k65qbkq0wv40000gn/T/ipykernel_18875/3778872193.py:57: DeprecationWarning: Migrate to using client.datasets.create_dataset via arize-phoenix-client
  dataset = phoenix_client.upload_dataset(
I0000 00:00:1764194329.580309  432990 fork_posix.cc:75] Other threads are currently calling into gRPC, skipping fork() handlers


running tasks |          | 0/5 (0.0%) | ⏳ 00:00<? | ?it/s

╭──────────────────────────────────────────── Crew Execution Started ─────────────────────────────────────────────╮
│                                                                                                                 │
│  Crew Execution Started                                                                                         │
│  Name: crew                                                                                                     │
│  ID: 508793c7-814b-4fdb-bbca-bc98ed0f9336                                                                       │
│  Tool Args:                                                                                                     │
│                                                                                                                 │
│                                                                                                                 │
╰─────────────────────────────────────────────────────────────────────────────────────────────────────────────────╯

╭─────────────────────────────────────────────── 🤖 Agent Started ────────────────────────────────────────────────╮
│                                                                                                                 │
│  Agent: Senior Tech Research Analyst                                                                            │
│                                                                                                                 │
│  Task: 1. Search for the absolute latest advancements in AI and data science advancements in 2024. 2. Identify  │
│  at least 5 distinct trends or technologies. 3. For each trend, find the specific company or research lab       │
│  responsible. 4. Collect URLs for every claim you make.                                                         │
│                                                                                                                 │
╰─────────────────────────────────────────────────────────────────────────────────────────────────────────────────╯

...

╭──────────────────────────────────────────────── Crew Completion ────────────────────────────────────────────────╮
│                                                                                                                 │
│  Crew Execution Completed                                                                                       │
│  Name: crew                                                                                                     │
│  ID: 996f5e57-11d3-48df-8407-3271a5959ae7                                                                       │
│  Tool Args:                                                                                                     │
│  Final Output: # The Renewable Revolution: Innovations Lighting the Way Forward                                 │
│                                                                                                                 │
│  The renewable energy sector is buzzing with innovation. From advanced solar technologies to the powerful       │
│  potential of green hydrogen, the landscape is shifting fast. In 2023, solutions are emerging that tackle       │
│  climate change head-on while promoting sustainability. Buckle up, because the future of energy is cleaner,     │
│  smarter, and incredibly exciting.                                                                              │
│                                                                                                                 │
│  ## Green Hydrogen Electrolyzers                                                                                │
│                                                                                                                 │
│  At the forefront of this energy revolution are green hydrogen electrolyzers. These innovations are enabling    │
│  the production of hydrogen from renewable sources more efficiently and at lower costs. This means cleaner      │
│  fuel for transport and industry, particularly in hard-to-decarbonize sectors. As electrolyzers become cheaper  │
│  and more efficient, green hydrogen will play a pivotal role in reshaping the energy landscape. Let's be        │
│  clear: this isn't just a trend. It's a game-changer for sustainable energy.                                    │
│                                                                                                                 │
│  ## Advanced Solar Panel Technology                                                                             │
│                                                                                                                 │
│  Next up is the rapid evolution of solar panel technology. Recent advancements in photovoltaic efficiency are   │
│  pushing the boundaries of what's possible. Today's solar panels can convert sunlight into energy more          │
│  effectively than ever before. This surge in efficiency is crucial for increasing global solar energy           │
│  adoption. As costs decrease and technology improves, harnessing the sun's power is becoming a viable option    │
│  for more individuals and businesses alike. It’s a bright spot in our quest for cleaner energy.                 │
│                                                                                                                 │
│  ## Carbon Capture and Storage (CCS)                                                                            │
│                                                                                                                 │
│  Then there's carbon capture and storage (CCS). This technology is being integrated with renewable energy       │
│  systems, creating a dual benefit: generating clean energy while actively reducing carbon emissions.            │
│  Essentially, CCS captures CO2 before it can enter the atmosphere and safely stores it. It’s a stabilizing      │
│  force against climate change. As we strive for net-ze

✅ Task runs completed.
🧠 Evaluation started.


running experiment evaluations |          | 0/10 (0.0%) | ⏳ 00:00<? | ?it/s


🔗 View this experiment: http://localhost:6006/datasets/RGF0YXNldDox/compare?experimentId=RXhwZXJpbWVudDox

Experiment Summary (11/26/25 04:07 PM -0600)
--------------------------------------------
| evaluator              |   n |   n_scores |   avg_score |
|:-----------------------|----:|-----------:|------------:|
| score_agent_sequence   |   5 |          5 |           0 |
| score_input_similarity |   5 |          5 |           0 |

Tasks Summary (11/26/25 04:07 PM -0600)
---------------------------------------
|   n_examples |   n_runs |   n_errors |
|-------------:|---------:|-----------:|
|            5 |        5 |          0 |


AttributeError: 'Client' object has no attribute 'url'